# Feature Matching and Object Detection

In [ ]:
from expt_utils import *
from sklearn.metrics import confusion_matrix

In [ ]:
img = plt.imread(f'{DS_DIR}/val/images/00049_3880.jpg')
img_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
img_edges = img_to_mexican_hat_edges(img_gray).astype(np.uint8)
gnd_trth_gray = cv.imread(f'{IMG_DIR}/ground_truth/00049_3880.jpg', cv.IMREAD_GRAYSCALE)


## SIFT Feature Matching

In [ ]:
sift = cv.SIFT_create()

keypoints_gray, descriptors_gray = sift.detectAndCompute(gnd_trth_gray, None)
keypoints_edge, descriptors_edge = sift.detectAndCompute(img_edges, None)

# object matching
bf = cv.BFMatcher()

matches = bf.knnMatch(descriptors_gray, descriptors_edge, k=2)

good_matches = []
total_correct = 0
for m, n in matches:
    if m.distance < 0.9 * n.distance: # if m distance (indicate to descripter to the grd_img) is less than 0.5 * n distance (indicate to descripter of the estimate_img) then it may belong to the object else it belong to the background
        good_matches.append(m)
        total_correct += 1

img_matches = cv.drawMatches(gnd_trth_gray, keypoints_gray, img_edges, keypoints_edge, good_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

ground_truth_keypoints = total_correct
predicted_keypoints = len(keypoints_edge)

ground_truth_labels = np.zeros(len(matches))
predicted_labels = np.zeros(len(matches))
for i, match in enumerate(matches):
    if i in [m.queryIdx for m in good_matches]:  # check if the index is in good_matches
        predicted_labels[i] = 1
    if i < len(good_matches):
        ground_truth_labels[i] = 1

conf_matrix = confusion_matrix(ground_truth_labels, predicted_labels)

print(f"accuracy: {ground_truth_keypoints / predicted_keypoints :.2%}")
print("Confusion Matrix:")
print(conf_matrix)
print("All Truth Keypoints", len(keypoints_gray))
print("All Estimated Keypoints", len(keypoints_edge))

plt.figure(figsize=(10,4), tight_layout=True)
plt.axis('off'), plt.title('SIFT Feature Matching')
plt.imshow(img_matches)
plt.savefig(f'{OUT_DIR}/07-01-sift_feature_matching')

## ORB Feature Matching

In [ ]:
orb = cv.ORB_create()

keypoints_gray, descriptors_gray = orb.detectAndCompute(gnd_trth_gray, None)
keypoints_edge, descriptors_edge = orb.detectAndCompute(img_edges, None)

# feature matching
bf = cv.BFMatcher()

matches = bf.knnMatch(descriptors_gray, descriptors_edge, k=2)

good_matches = []
total_correct = 0
for m, n in matches:
    if m.distance < 0.9 * n.distance:
        good_matches.append(m)
        total_correct += 1

img_matches = cv.drawMatches(gnd_trth_gray, keypoints_gray, img_edges, keypoints_edge, good_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

ground_truth_labels = np.zeros(len(matches))
predicted_labels = np.zeros(len(matches))
for i, match in enumerate(matches):
    if i in [m.queryIdx for m in good_matches]:  # check if the index is in good_matches
        predicted_labels[i] = 1
    if i < len(good_matches):
        ground_truth_labels[i] = 1

conf_matrix = confusion_matrix(ground_truth_labels, predicted_labels)

print(f"accuracy: {ground_truth_keypoints / predicted_keypoints :.2%}")
print("Confusion Matrix:")
print(conf_matrix)
print("All Truth Keypoints", len(keypoints_gray))
print("All Estimated Keypoints", len(keypoints_edge))


plt.figure(figsize=(10, 4), tight_layout=True)
plt.axis('off'), plt.title('ORB Feature Matching')
plt.imshow(img_matches)
plt.savefig(f'{OUT_DIR}/07-02-orb_feature_matching')